In [5]:
import torch
annealing_steps = 10

i = 0
for t in torch.linspace(0, 1, annealing_steps)[1:]:
  i += 1
  print(t)
print(i) 

ts = torch.zeros(9)
print(ts)

tensor(0.1111)
tensor(0.2222)
tensor(0.3333)
tensor(0.4444)
tensor(0.5556)
tensor(0.6667)
tensor(0.7778)
tensor(0.8889)
tensor(1.)
9
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.])
